In [1]:

import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
import datetime
from datetime import date, timedelta, datetime
import math
import matplotlib as mpl
import numpy as np
import talib as ta
import yfinance as yf
from os import walk


In [2]:
def trade(stock,startDate,endDate): 
    StartingAmount = 500_000/505

    df = pd.read_csv("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/filter_30y_stock_csvs/{}.csv".format(stock))
    df = df.rename(columns={"CLOSE": "Adj Close", "VOLUME": "Volume"})

    df['Datetime'] = pd.to_datetime(df['Datetime'])

    df = df[~(df['Datetime'] < startDate)]
    df = df[~(df['Datetime'] > endDate)]



    PL=0.00
    Total_Gain =0
    starting_price =1
    #PG -> Per_Gain

    Start_Price = (df['Adj Close'].head(1))
    Start_Price = float(Start_Price)

    End_Price = (df['Adj Close'].tail(1))
    End_Price = float(End_Price)

    Return = (PL/Start_Price)
    Return_Per = "{:.2%}".format(Return)



    benchRe = End_Price - Start_Price
    benchREP = (benchRe/Start_Price) 
    TotalPro = (benchREP) * StartingAmount

    TotalReturn = StartingAmount
    index =0
    bought = False
    
    #ADX
        #Tells how strong the trend is
            #Anything above 30 is a strong trend

    #RSI
        #Tells if there is an over buy or over sell
            #From 0-30
                #Over sell
            #From 30-70
                #Normal
            #70-100
                #Over buy

    # for date_var,row in df.iterrows():
    #     maxValue = df.iloc[index-5:index]['Adj Close'].max()
    #     if math.isnan(maxValue):
    #         maxValue = 0

    #     if(df.iloc[index-1]['ADX']>22):
    #         if not bought and df.iloc[index-1]['RSI'] > 25:
    #             if df.iloc[index-1]['Volume Higher than MAVG']==1 and df.iloc[index-1]['Price Lower than MAVG']==1:
    #                     # print(row['Datetime'],row['Adj Close'], '- Buy due to Volume + price')
    #                     close_adj = row['Adj Close']
    #                     starting_price = close_adj
    #                     bought = True
    #             elif df.iloc[index-1]['Adj Close'] > maxValue and maxValue != 0:
    #                 # print(row['Datetime'],row['Adj Close'], '- Buy due to going above threshhold')
    #                 close_adj = row['Adj Close']
    #                 starting_price = close_adj
    #                 bought = True
    #         else:
    #             if bought:
    #                 if (df.iloc[index-1]['Volume Higher than MAVG']==0 or df.iloc[index-1]['RSI'] < 75) and df.iloc[index-1]['Price Lower than MAVG']==0 :
    #                         #print(row['Datetime'],row['Adj Close'],'- Sell due to volume change')
    #                         close_adj = row['Adj Close']
    #                         single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
    #                         Total_Gain += single_trade_percent_gain
    #                         # print("This trade gain/loss results: "+str(round(single_trade_percent_gain*100,2))+"%")
    #                         # print()
    #                         bought = False

    #                 elif df.iloc[index-1]['Adj Close'] < maxValue and maxValue != 0 : 
    #                         #print(row['Datetime'],row['Adj Close'],'- Sell due to going below threshhold')
    #                         close_adj = row['Adj Close']
    #                         single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
    #                         Total_Gain += single_trade_percent_gain
    #                         # print("This trade gain/loss results: "+str(round(single_trade_percent_gain*100,2))+"%")
    #                         # print()
    #                         bought = False
    #     index +=1
    #     TotalReturn = (1 + Total_Gain) * TotalReturn
    #     Total_Gain = 0
    # CONVERT EVERYTHING INTO NUMPY
    for date_var,row in df.iterrows():
        maxValue = df.iloc[index-5:index]['Adj Close'].max()
        if math.isnan(maxValue):
            maxValue = 0

    

        #print("This is the price: ${} with maxVal of: ${} with adx of {} on the date of: {} ".format(row['Adj Close'],maxValue,df.iloc[index-1]['ADX'],row['Datetime']))

        if(df.iloc[index-1]['ADX']>15): #If trend is appearing - See if its a good time to buy
            if not bought:
                # if df.iloc[index-1]['Volume Higher than MAVG']==1 and df.iloc[index-1]['Price Lower than MAVG']==1: 
                        
                #         close_adj = row['Adj Close']
                #         starting_price = close_adj
                #         bought = True

                # if df.iloc[index-1]['Adj Close'] > maxValue and maxValue != 0: #This doesnt work -- edit and relook
                    
                #     close_adj = row['Adj Close']
                #     starting_price = close_adj
                #     bought = True

                if df.iloc[index-1]['RSI'] > 70:
                    
                    close_adj = row['Adj Close']
                    starting_price = close_adj
                    bought = True  

        if bought and close_adj != row['Adj Close']: #This is sell clause
            if (df.iloc[index-1]['Volume Higher than MAVG']==0 or df.iloc[index-1]['RSI'] < 35) and df.iloc[index-1]['Price Lower than MAVG']==0 :
                   
                    close_adj = row['Adj Close']
                    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
                    Total_Gain += single_trade_percent_gain
                    
                    bought = False
                    
            elif (((df.iloc[index-1]['Adj Close']- starting_price)/starting_price) * 100) > (.5 * 100):
                   
                    close_adj = row['Adj Close']
                    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
                    Total_Gain += single_trade_percent_gain
                    
                    bought = False

                    

            elif df.iloc[index-1]['Adj Close'] < maxValue and maxValue != 0: 
                    
                    close_adj = row['Adj Close']
                    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
                    Total_Gain += single_trade_percent_gain
                    
                    bought = False

        index +=1
        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0


    
    if bought:
        
        close_adj = row['Adj Close']
        single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
        Total_Gain += single_trade_percent_gain
        
        TotalReturn = (1+Total_Gain)*TotalReturn
        Total_Gain = 0

    Hold_Return = (End_Price - Start_Price)
    Hold_Return_Per = "{:.2%}".format((End_Price-Start_Price)/Start_Price)
    #TotalReturn = (Total_Gain+StartingAmount)
    Total_Gain = ((TotalReturn - StartingAmount)/StartingAmount) 
    return round(TotalReturn-StartingAmount,2)


In [5]:
f = []
ROIall = {}
money = {}
startDate = datetime.strptime('2020-09-01',"%Y-%m-%d")
offsetDate = timedelta(39)
endDate = datetime.strptime('2020-11-01',"%Y-%m-%d")
sum = 0
startAmt = 500_000/505
for (dirpath, dirnames, filenames) in walk("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/clean_30y_stock_csvs/"):
    f.extend(filenames)
    break


for stock in f:
    stock = stock[:-4]
    money[stock] = trade(stock,startDate - offsetDate,endDate)
    print("{} is done with profit of: {} which is a percent of: {}%".format(stock,money[stock],round((money[stock]/startAmt)*100,2)))

for s in money:
    sum+= money[s]

percentageOfWin = ((sum)/500_000) * 100
print("Total win of ${} with % of {}%".format(round(sum,2),round(percentageOfWin,2)))


A is done with profit of: -17.42 which is a percent of: -1.76%
AAL is done with profit of: 0.0 which is a percent of: 0.0%
AAP is done with profit of: -9.61 which is a percent of: -0.97%
AAPL is done with profit of: 36.48 which is a percent of: 3.68%
ABBV is done with profit of: 0.0 which is a percent of: 0.0%
ABC is done with profit of: 0.0 which is a percent of: 0.0%
ABMD is done with profit of: 42.15 which is a percent of: 4.26%
ABT is done with profit of: 6.33 which is a percent of: 0.64%
ACN is done with profit of: -24.78 which is a percent of: -2.5%
ADBE is done with profit of: -36.93 which is a percent of: -3.73%
ADI is done with profit of: 0.0 which is a percent of: 0.0%
ADM is done with profit of: 35.94 which is a percent of: 3.63%
ADP is done with profit of: -83.71 which is a percent of: -8.45%
ADSK is done with profit of: -9.9 which is a percent of: -1.0%
AEE is done with profit of: 0.0 which is a percent of: 0.0%
AEP is done with profit of: 45.32 which is a percent of: 4.58

In [4]:
sp500 = yf.download('^GSPC',startDate,endDate)
profitsp500percentage = ((sp500['Close'][-1] - sp500['Close'][0])/sp500['Close'][0])*100
profitsp500 = (500_000) * (profitsp500percentage/100)

print("This is SAP 500 : {}% and the increase in money: {}".format(round(profitsp500percentage,2),round(profitsp500,2)))

[*********************100%***********************]  1 of 1 completed
This is SAP 500 : -7.28% and the increase in money: -36392.89
